<a href="https://colab.research.google.com/github/alion0761/Deneme/blob/main/Otomatik_Proje_Olu%C5%9Fturucu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import json

# Proje Adı
project_name = "hollandali-translator"

# Dosya İçerikleri
files = {
    "package.json": json.dumps({
        "name": "hollandali-translator",
        "version": "0.1.0",
        "private": True,
        "scripts": {
            "dev": "next dev",
            "build": "next build",
            "start": "next start",
            "lint": "next lint"
        },
        "dependencies": {
            "react": "^18",
            "react-dom": "^18",
            "next": "14.2.0",
            "lucide-react": "^0.378.0"
        },
        "devDependencies": {
            "typescript": "^5",
            "@types/node": "^20",
            "@types/react": "^18",
            "@types/react-dom": "^18",
            "postcss": "^8",
            "tailwindcss": "^3.4.1",
            "eslint": "^8",
            "eslint-config-next": "14.2.0"
        }
    }, indent=2),

    "tsconfig.json": json.dumps({
        "compilerOptions": {
            "lib": ["dom", "dom.iterable", "esnext"],
            "allowJs": True,
            "skipLibCheck": True,
            "strict": True,
            "noEmit": True,
            "esModuleInterop": True,
            "module": "esnext",
            "moduleResolution": "bundler",
            "resolveJsonModule": True,
            "isolatedModules": True,
            "jsx": "preserve",
            "incremental": True,
            "plugins": [{"name": "next"}],
            "paths": {"@/*": ["./*"]}
        },
        "include": ["next-env.d.ts", "**/*.ts", "**/*.tsx", ".next/types/**/*.ts"],
        "exclude": ["node_modules"]
    }, indent=2),

    "next.config.mjs": """/** @type {import('next').NextConfig} */
const nextConfig = {};

export default nextConfig;
""",

    "tailwind.config.ts": """import type { Config } from "tailwindcss";

const config: Config = {
  content: [
    "./pages/**/*.{js,ts,jsx,tsx,mdx}",
    "./components/**/*.{js,ts,jsx,tsx,mdx}",
    "./app/**/*.{js,ts,jsx,tsx,mdx}",
  ],
  theme: {
    extend: {
      backgroundImage: {
        "gradient-radial": "radial-gradient(var(--tw-gradient-stops))",
        "gradient-conic":
          "conic-gradient(from 180deg at 50% 50%, var(--tw-gradient-stops))",
      },
    },
  },
  plugins: [],
};
export default config;
""",

    "postcss.config.js": """module.exports = {
  plugins: {
    tailwindcss: {},
    autoprefixer: {},
  },
};
""",

    "next-env.d.ts": """/// <reference types="next" />
/// <reference types="next/image-types/global" />

// NOTE: This file should not be edited
// see https://nextjs.org/docs/basic-features/typescript for more information.
""",

    ".gitignore": """# dependencies
/node_modules
/.pnp
.pnp.js

# testing
/coverage

# next.js
/.next/
/out/

# production
/build

# misc
.DS_Store
*.pem

# debug
npm-debug.log*
yarn-debug.log*
yarn-error.log*

# local env files
.env*.local

# vercel
.vercel
""",

    ".env.local.example": """# Google Gemini API Anahtarınızı buraya yapıştırın
# Bu dosyanın adını .env.local olarak değiştirin ve API anahtarınızı ekleyin
NEXT_PUBLIC_GEMINI_API_KEY=
""",

    "README.md": """# HollandAli - Simultane Çeviri Uygulaması

## Kurulum
1. Terminali açın: `npm install`
2. `.env.local.example` dosyasının adını `.env.local` yapın ve API anahtarınızı girin.
3. Çalıştırın: `npm run dev`
""",

    "app/globals.css": """@tailwind base;
@tailwind components;
@tailwind utilities;

:root {
  --foreground-rgb: 0, 0, 0;
  --background-start-rgb: 214, 219, 220;
  --background-end-rgb: 255, 255, 255;
}

body {
  color: rgb(var(--foreground-rgb));
  background: linear-gradient(to bottom, transparent, rgb(var(--background-end-rgb))) rgb(var(--background-start-rgb));
}
""",

    "app/layout.tsx": """import type { Metadata } from "next";
import "./globals.css";

export const metadata: Metadata = {
  title: "HollandAli - Simultane Çeviri",
  description: "AI tabanlı anlık sesli çeviri uygulaması",
};

export default function RootLayout({
  children,
}: Readonly<{
  children: React.ReactNode;
}>) {
  return (
    <html lang="tr">
      <body>{children}</body>
    </html>
  );
}
""",

    "app/page.tsx": """import TranslatorApp from "@/components/TranslatorApp";

export default function Home() {
  return (
    <main className="flex min-h-screen flex-col items-center justify-between p-0 md:p-24 bg-slate-50">
      <TranslatorApp />
    </main>
  );
}
""",

    "components/TranslatorApp.tsx": """\"use client\";

import React, { useState, useEffect, useRef } from 'react';
import { Mic, MicOff, Languages, Volume2, VolumeX, History, Trash2, Settings, ArrowRightLeft, Copy, Check, Loader2, Sparkles, X } from 'lucide-react';

export default function TranslatorApp() {
  const [isRecording, setIsRecording] = useState(false);
  const [sourceText, setSourceText] = useState('');
  const [translatedText, setTranslatedText] = useState('');

  const [languagePair, setLanguagePair] = useState({ src: 'tr-TR', dest: 'en-US', srcLabel: 'Türkçe', destLabel: 'English' });

  const [history, setHistory] = useState<{src: string, dest: string, id: number}[]>([]);
  const [isCopied, setIsCopied] = useState(false);
  const [activeTab, setActiveTab] = useState('translate');
  const [isTranslating, setIsTranslating] = useState(false);
  const [autoPlay, setAutoPlay] = useState(true);
  const [isAutoMode, setIsAutoMode] = useState(false);
  const [availableVoices, setAvailableVoices] = useState<SpeechSynthesisVoice[]>([]);

  const [showSettings, setShowSettings] = useState(false);

  const recognitionRef = useRef<any>(null);
  const synthesisRef = useRef<SpeechSynthesis | null>(null);

  const sourceTextRef = useRef(sourceText);
  const isRecordingRef = useRef(isRecording);
  const autoPlayRef = useRef(autoPlay);
  const isAutoModeRef = useRef(isAutoMode);
  const languagePairRef = useRef(languagePair);

  const LANGUAGES = [
    { code: 'tr-TR', label: 'Türkçe', flagCode: 'tr' },
    { code: 'en-US', label: 'English', flagCode: 'us' },
    { code: 'ar-SA', label: 'العربية', flagCode: 'sa' },
    { code: 'nl-NL', label: 'Nederlands', flagCode: 'nl' },
    { code: 'ru-RU', label: 'Русский', flagCode: 'ru' }
  ];

  const getFlagUrl = (langCode: string) => {
    if (langCode.startsWith('tr')) return 'https://flagcdn.com/w80/tr.png';
    if (langCode.startsWith('en')) return 'https://flagcdn.com/w80/us.png';
    if (langCode.startsWith('ar')) return 'https://flagcdn.com/w80/sa.png';
    if (langCode.startsWith('nl')) return 'https://flagcdn.com/w80/nl.png';
    if (langCode.startsWith('ru')) return 'https://flagcdn.com/w80/ru.png';
    return 'https://flagcdn.com/w80/un.png';
  };

  const isRTL = (langCode: string) => langCode.startsWith('ar');

  useEffect(() => { sourceTextRef.current = sourceText; }, [sourceText]);
  useEffect(() => { isRecordingRef.current = isRecording; }, [isRecording]);
  useEffect(() => { autoPlayRef.current = autoPlay; }, [autoPlay]);
  useEffect(() => { isAutoModeRef.current = isAutoMode; }, [isAutoMode]);
  useEffect(() => { languagePairRef.current = languagePair; }, [languagePair]);

  useEffect(() => {
    if (typeof window !== 'undefined' && window.speechSynthesis) {
        const updateVoices = () => {
            const voices = window.speechSynthesis.getVoices();
            setAvailableVoices(voices);
        };
        updateVoices();
        window.speechSynthesis.onvoiceschanged = updateVoices;
        return () => { window.speechSynthesis.onvoiceschanged = null; };
    }
  }, []);

  useEffect(() => {
    if (typeof window !== 'undefined') {
      const SpeechRecognition = (window as any).SpeechRecognition || (window as any).webkitSpeechRecognition;

      if (SpeechRecognition) {
        const recognition = new SpeechRecognition();
        recognition.continuous = true;
        recognition.interimResults = true;
        recognition.lang = languagePair.src;

        recognition.onresult = (event: any) => {
          let interimTranscript = '';
          let finalTranscript = '';

          for (let i = event.resultIndex; i < event.results.length; ++i) {
            if (event.results[i].isFinal) {
              finalTranscript += event.results[i][0].transcript;
            } else {
              interimTranscript += event.results[i][0].transcript;
            }
          }

          if (finalTranscript) {
            setSourceText(finalTranscript);
            setTranslatedText('');
            translateWithGemini(finalTranscript);
          }
          else if (interimTranscript) {
             setSourceText(interimTranscript);
          }
        };

        recognition.onerror = (event: any) => {
          if (event.error !== 'aborted') { console.error('Speech recognition error', event.error); }
          if (event.error === 'not-allowed') { setIsRecording(false); }
        };

        recognition.onend = () => {
          if (isRecordingRef.current) {
             setTimeout(() => {
                try { recognition.start(); } catch (err) { console.log(\"Recognition restart skipped\"); }
             }, 300);
          }
        };

        recognitionRef.current = recognition;
      }

      synthesisRef.current = window.speechSynthesis;

      return () => {
        if (recognitionRef.current) {
          recognitionRef.current.onend = null;
          recognitionRef.current.stop();
        }
        if (synthesisRef.current) {
          synthesisRef.current.cancel();
        }
      };
    }
  }, [languagePair.src]);

  const toggleRecording = () => {
    if (isRecording) {
      setIsRecording(false);
      recognitionRef.current?.stop();
      synthesisRef.current?.cancel();
    } else {
      setSourceText('');
      setTranslatedText('');
      setIsRecording(true);
      try { recognitionRef.current?.start(); } catch (e) { console.error(\"Start error:\", e); }
    }
  };

  const swapLanguages = () => {
    if (isRecording) { setIsRecording(false); recognitionRef.current?.stop(); }
    synthesisRef.current?.cancel();
    setLanguagePair(prev => ({
      src: prev.dest, dest: prev.src, srcLabel: prev.destLabel, destLabel: prev.srcLabel
    }));
  };

  const translateWithGemini = async (textToTranslate: string) => {
    if (!textToTranslate.trim()) return;

    setIsTranslating(true);
    const apiKey = process.env.NEXT_PUBLIC_GEMINI_API_KEY || \"\";

    if (!apiKey) {
      alert(\"Lütfen .env.local dosyasına API anahtarınızı ekleyin.\");
      setIsTranslating(false);
      return;
    }

    let prompt = \"\";

    if (isAutoModeRef.current) {
      prompt = `
        You are an intelligent simultaneous interpreter with language detection capabilities.

        Context:
        - The Speech-to-Text engine is currently listening in: ${languagePairRef.current.srcLabel} (${languagePairRef.current.src}).
        - The Target Language is: ${languagePairRef.current.destLabel} (${languagePairRef.current.dest}).

        Input Text (from STT): \"${textToTranslate}\"

        Task:
        1. Analyze the Input Text. Does it look like ${languagePairRef.current.srcLabel}, OR does it look like ${languagePairRef.current.destLabel} (phonetically transcribed incorrectly)?
        2. If it is ${languagePairRef.current.srcLabel}: Translate it to ${languagePairRef.current.destLabel}.
        3. If it looks like ${languagePairRef.current.destLabel}: Reconstruct the original sentence in ${languagePairRef.current.destLabel}, and then translate it to ${languagePairRef.current.srcLabel}.

        Response JSON format:
        {
          \"detectedLanguage\": \"CODE (e.g. tr-TR, en-US, ar-SA, nl-NL, ru-RU)\",
          \"originalTextCorrected\": \"The corrected version of the input text\",
          \"translatedText\": \"The final translation\"
        }
      `;
    } else {
      prompt = `
        Translate this text from ${languagePairRef.current.srcLabel} to ${languagePairRef.current.destLabel}.
        Input: \"${textToTranslate}\"
        Response JSON format:
        {
          \"detectedLanguage\": \"${languagePairRef.current.src}\",
          \"originalTextCorrected\": \"${textToTranslate}\",
          \"translatedText\": \"Translation\"
        }
      `;
    }

    try {
      const response = await fetch(
        `https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent?key=${apiKey}`,
        {
          method: 'POST',
          headers: { 'Content-Type': 'application/json' },
          body: JSON.stringify({
            contents: [{ parts: [{ text: prompt }] }],
            generationConfig: { responseMimeType: \"application/json\" }
          })
        }
      );

      const data = await response.json();
      const rawText = data.candidates?.[0]?.content?.parts?.[0]?.text;

      if (rawText) {
        const result = JSON.parse(rawText);
        const translatedPart = result.translatedText;
        const correctedSource = result.originalTextCorrected;
        const detectedLang = result.detectedLanguage;

        setTranslatedText(translatedPart);

        if (isAutoModeRef.current && correctedSource !== textToTranslate) {
           setSourceText(correctedSource);
        }

        addToHistory(correctedSource || textToTranslate, translatedPart);

        if (autoPlayRef.current) {
          let langToSpeak = languagePairRef.current.dest;
          if (isAutoModeRef.current) {
             if (detectedLang.includes(languagePairRef.current.dest.split('-')[0])) {
               langToSpeak = languagePairRef.current.src;
             }
          }
          speakText(translatedPart, langToSpeak);
        }
      }
    } catch (error) {
      console.error(\"Gemini Translation Error:\", error);
    } finally {
      setIsTranslating(false);
    }
  };

  const addToHistory = (src: string, dest: string) => {
    setHistory(prev => [{ src, dest, id: Date.now() }, ...prev]);
  };

  const speakText = (text: string, langCode: string) => {
    if (!synthesisRef.current || !text) return;

    synthesisRef.current.cancel();

    const utterance = new SpeechSynthesisUtterance(text);
    utterance.lang = langCode;
    utterance.rate = 0.9;

    if (availableVoices.length > 0) {
        let selectedVoice = availableVoices.find(v => v.lang === langCode);

        if (!selectedVoice) {
            const genericLang = langCode.split('-')[0];
            selectedVoice = availableVoices.find(v => v.lang.startsWith(genericLang));
        }

        if (!selectedVoice && langCode.startsWith('ar')) {
             selectedVoice = availableVoices.find(v => v.name.toLowerCase().includes('arabic') || v.name.toLowerCase().includes('arab'));
        }

        if (selectedVoice) {
            utterance.voice = selectedVoice;
            utterance.lang = selectedVoice.lang;
        }
    }

    synthesisRef.current.speak(utterance);
  };

  const copyToClipboard = (text: string) => {
    const textArea = document.createElement(\"textarea\");
    textArea.value = text;
    textArea.style.position = \"fixed\";
    textArea.style.left = \"-9999px\";
    textArea.style.top = \"0\";
    document.body.appendChild(textArea);
    textArea.focus();
    textArea.select();
    try {
      const successful = document.execCommand('copy');
      if (successful) {
        setIsCopied(true);
        setTimeout(() => setIsCopied(false), 2000);
      }
    } catch (err) { console.error('Copy failed', err); }
    document.body.removeChild(textArea);
  };

  const clearAll = () => {
    setSourceText('');
    setTranslatedText('');
    synthesisRef.current?.cancel();
  };

  return (
    <div className=\"min-h-screen bg-slate-50 text-slate-900 font-sans md:max-w-md md:mx-auto md:shadow-2xl md:border-x border-slate-200 flex flex-col relative overflow-hidden\">

      {showSettings && (
        <div className=\"absolute inset-0 z-50 bg-black/50 backdrop-blur-sm flex items-end sm:items-center justify-center p-4\">
          <div className=\"bg-white w-full max-w-sm rounded-3xl p-6 shadow-2xl animate-in fade-in slide-in-from-bottom-10 duration-300\">
            <div className=\"flex justify-between items-center mb-6\">
              <h3 className=\"text-xl font-bold text-slate-800\">Dil Ayarları</h3>
              <button onClick={() => setShowSettings(false)} className=\"p-2 bg-slate-100 rounded-full hover:bg-slate-200 transition-colors\">
                <X size={20} className=\"text-slate-600\" />
              </button>
            </div>

            <div className=\"mb-6\">
              <label className=\"text-xs font-bold text-slate-400 uppercase tracking-wider mb-3 block\">Konuşulan Dil (Kaynak)</label>
              <div className=\"grid grid-cols-3 gap-2\">
                {LANGUAGES.map(lang => (
                  <button
                    key={lang.code}
                    onClick={() => {
                      setLanguagePair(prev => ({...prev, src: lang.code, srcLabel: lang.label}));
                    }}
                    className={`flex flex-col items-center gap-2 p-3 rounded-xl border transition-all ${languagePair.src === lang.code ? 'border-blue-500 bg-blue-50 ring-2 ring-blue-200' : 'border-slate-100 hover:bg-slate-50'}`}
                  >
                    <img src={getFlagUrl(lang.code)} alt={lang.label} className=\"w-8 h-6 object-cover rounded shadow-sm\" />
                    <span className=\"text-xs font-medium text-slate-700\">{lang.label}</span>
                  </button>
                ))}
              </div>
            </div>

            <div>
              <label className=\"text-xs font-bold text-slate-400 uppercase tracking-wider mb-3 block\">Çevrilen Dil (Hedef)</label>
              <div className=\"grid grid-cols-3 gap-2\">
                {LANGUAGES.map(lang => (
                  <button
                    key={lang.code}
                    onClick={() => {
                      setLanguagePair(prev => ({...prev, dest: lang.code, destLabel: lang.label}));
                    }}
                    className={`flex flex-col items-center gap-2 p-3 rounded-xl border transition-all ${languagePair.dest === lang.code ? 'border-purple-500 bg-purple-50 ring-2 ring-purple-200' : 'border-slate-100 hover:bg-slate-50'}`}
                  >
                    <img src={getFlagUrl(lang.code)} alt={lang.label} className=\"w-8 h-6 object-cover rounded shadow-sm\" />
                    <span className=\"text-xs font-medium text-slate-700\">{lang.label}</span>
                  </button>
                ))}
              </div>
            </div>

            <button onClick={() => setShowSettings(false)} className=\"w-full mt-8 bg-blue-600 hover:bg-blue-700 text-white py-3 rounded-xl font-semibold shadow-lg shadow-blue-200 active:scale-95 transition-all\">
              Tamam
            </button>
          </div>
        </div>
      )}

      <header className=\"bg-white p-4 shadow-sm z-10 flex justify-between items-center sticky top-0\">
        <div className=\"flex items-center gap-2\">
          <div className=\"bg-blue-600 text-white p-2 rounded-lg\">
            <Languages size={20} />
          </div>
          <h1 className=\"font-bold text-lg text-slate-800\">HollandAli</h1>
        </div>
        <button
          onClick={() => setActiveTab(activeTab === 'translate' ? 'history' : 'translate')}
          className={`p-2 rounded-full transition-colors ${activeTab === 'history' ? 'bg-blue-100 text-blue-600' : 'text-slate-500 hover:bg-slate-100'}`}
        >
          <History size={24} />
        </button>
      </header>

      <main className=\"flex-1 overflow-y-auto pb-40 scroll-smooth\">
        {activeTab === 'history' ? (
          <div className=\"p-4 space-y-4\">
            <h2 className=\"text-sm font-semibold text-slate-500 mb-2\">Çeviri Geçmişi</h2>
            {history.length === 0 ? (
              <div className=\"text-center py-10 text-slate-400\">
                <History size={48} className=\"mx-auto mb-2 opacity-20\" />
                <p>Henüz çeviri yok.</p>
              </div>
            ) : (
              history.map((item) => (
                <div key={item.id} className=\"bg-white p-4 rounded-xl shadow-sm border border-slate-100\">
                  <p className={`text-slate-800 mb-1 ${isRTL(languagePair.src) ? 'text-right' : 'text-left'}`}>{item.src}</p>
                  <p className={`text-blue-600 font-medium ${isRTL(languagePair.dest) ? 'text-right' : 'text-left'}`}>{item.dest}</p>
                </div>
              ))
            )}
            {history.length > 0 && (
              <button onClick={() => setHistory([])} className=\"w-full py-3 text-red-500 text-sm font-medium hover:bg-red-50 rounded-lg transition-colors\">
                Geçmişi Temizle
              </button>
            )}
          </div>
        ) : (
          <div className=\"p-4 space-y-6\">

            <div className=\"flex justify-center\">
              <button
                onClick={() => setIsAutoMode(!isAutoMode)}
                className={`flex items-center gap-2 px-4 py-2 rounded-full text-sm font-semibold transition-all duration-300 shadow-sm border ${
                  isAutoMode
                    ? 'bg-purple-100 text-purple-700 border-purple-200 ring-2 ring-purple-100'
                    : 'bg-white text-slate-500 border-slate-200 hover:bg-slate-50'
                }`}
              >
                <Sparkles size={16} className={isAutoMode ? \"fill-purple-500 animate-pulse\" : \"\"} />
                {isAutoMode ? 'Otomatik Algılama AÇIK' : 'Otomatik Algılama (Sohbet)'}
              </button>
            </div>

            <div className=\"bg-white rounded-2xl p-5 shadow-sm border border-slate-100 relative group transition-all duration-300 hover:shadow-md\">
              <div className=\"flex justify-between items-center mb-2\">
                <span className=\"text-xs font-bold text-slate-400 uppercase tracking-wider flex items-center gap-2\">
                  <img src={getFlagUrl(languagePair.src)} alt={languagePair.src} className=\"w-6 h-4 object-cover rounded shadow-sm\" />
                  {languagePair.srcLabel} (Duyulan)
                </span>
                {sourceText && (
                  <button onClick={clearAll} className=\"text-slate-300 hover:text-red-400 p-1\">
                    <Trash2 size={16} />
                  </button>
                )}
              </div>
              <div className={`min-h-[80px] text-lg font-medium text-slate-700 leading-relaxed ${isRTL(languagePair.src) ? 'text-right' : 'text-left'}`} dir={isRTL(languagePair.src) ? 'rtl' : 'ltr'}>
                {sourceText || <span className=\"text-slate-300 italic\">Konuşmaya başlamak için mikrofona dokunun...</span>}
              </div>
            </div>

            <div className={`rounded-2xl p-5 shadow-lg relative overflow-hidden transition-all duration-300 ${isAutoMode ? 'bg-purple-600 shadow-purple-200' : 'bg-blue-600 shadow-blue-200'}`}>
              <div className=\"absolute -top-10 -right-10 w-32 h-32 bg-white rounded-full opacity-10 blur-xl pointer-events-none\"></div>

              <div className=\"flex justify-between items-center mb-2 relative z-10\">
                <div className=\"flex items-center gap-2\">
                  <span className={`text-xs font-bold uppercase tracking-wider flex items-center gap-2 ${isAutoMode ? 'text-purple-200' : 'text-blue-200'}`}>
                    <img src={getFlagUrl(languagePair.dest)} alt={languagePair.dest} className=\"w-6 h-4 object-cover rounded shadow-sm\" />
                    {languagePair.destLabel} (Çeviri)
                  </span>
                  {isTranslating && <Loader2 size={14} className=\"text-white/70 animate-spin\" />}
                </div>

                <div className=\"flex gap-2\">
                  <button
                    onClick={() => setAutoPlay(!autoPlay)}
                    className={`flex items-center gap-1 px-2 py-1 rounded-md text-[10px] font-bold transition-colors mr-2 border ${
                      autoPlay
                      ? 'bg-white/20 text-white border-transparent'
                      : 'bg-transparent text-white/50 border-white/30 hover:bg-white/10'
                    }`}
                  >
                    {autoPlay ? <Volume2 size={12} /> : <VolumeX size={12} />}
                    {autoPlay ? 'OTO' : 'SESSİZ'}
                  </button>

                  <button
                    onClick={() => copyToClipboard(translatedText)}
                    className=\"text-white/70 hover:text-white p-1.5 hover:bg-white/20 rounded-md transition-colors\"
                  >
                    {isCopied ? <Check size={16} /> : <Copy size={16} />}
                  </button>
                  <button
                    onClick={() => speakText(translatedText, languagePair.dest)}
                    className=\"text-white/70 hover:text-white p-1.5 hover:bg-white/20 rounded-md transition-colors\"
                  >
                    <Volume2 size={18} />
                  </button>
                </div>
              </div>
              <div className={`min-h-[100px] text-xl font-semibold text-white leading-relaxed relative z-10 ${isRTL(languagePair.dest) ? 'text-right' : 'text-left'}`} dir={isRTL(languagePair.dest) ? 'rtl' : 'ltr'}>
                {translatedText || <span className=\"text-white/40 italic\">Çeviri bekleniyor...</span>}
              </div>
            </div>

            {isRecording && (
              <div className=\"flex justify-center items-center gap-1 h-12\">
                {[...Array(5)].map((_, i) => (
                  <div key={i} className=\"w-1.5 bg-red-500 rounded-full animate-pulse\" style={{ height: `${Math.random() * 20 + 10}px`, animationDuration: `${Math.random() * 0.5 + 0.5}s` }}></div>
                ))}
                <span className=\"text-xs font-medium text-red-500 ml-2 animate-pulse\">
                  {isAutoMode ? 'HER İKİ DİL DİNLENİYOR...' : 'DİNLİYOR...'}
                </span>
              </div>
            )}
          </div>
        )}
      </main>

      <div className=\"absolute bottom-0 left-0 right-0 bg-white/90 backdrop-blur-md p-6 border-t border-slate-100 rounded-t-3xl shadow-[0_-5px_20px_-5px_rgba(0,0,0,0.1)]\">
        <div className=\"flex justify-between items-center max-w-sm mx-auto\">

          <div className=\"flex flex-col items-center gap-1 w-1/4\">
            <button
              onClick={swapLanguages}
              className={`p-3 rounded-full transition-colors ${isAutoMode ? 'bg-slate-100 text-slate-300 cursor-not-allowed' : 'hover:bg-slate-100 text-slate-600'}`}
              disabled={isAutoMode || isRecording}
            >
              <ArrowRightLeft size={20} />
            </button>
            <span className=\"text-base font-bold text-slate-500 flex items-center justify-center gap-2 mt-1\">
               <img src={getFlagUrl(languagePair.src)} className=\"w-4 h-3 object-cover rounded-sm\" alt=\"src\" />
               {!isAutoMode && <span>&rarr;</span>}
               {isAutoMode && <Sparkles size={12} className=\"text-purple-500\" />}
               <img src={getFlagUrl(languagePair.dest)} className=\"w-4 h-3 object-cover rounded-sm\" alt=\"dest\" />
            </span>
          </div>

          <div className=\"flex justify-center -mt-8 relative z-20 w-1/4\">
            <div className={`absolute inset-0 bg-blue-500 rounded-full blur-xl opacity-20 transition-opacity duration-300 ${isRecording ? 'opacity-60 scale-150' : 'scale-100'}`}></div>
            <button
              onClick={toggleRecording}
              className={`
                relative flex items-center justify-center w-20 h-20 rounded-full shadow-xl transition-all duration-300 transform active:scale-95
                ${isRecording
                  ? 'bg-red-500 text-white shadow-red-200 ring-4 ring-red-100'
                  : isAutoMode
                    ? 'bg-purple-600 text-white shadow-purple-200 hover:bg-purple-700 ring-4 ring-purple-50'
                    : 'bg-blue-600 text-white shadow-blue-200 hover:bg-blue-700 ring-4 ring-blue-50'
                }
              `}
            >
              {isRecording ? <MicOff size={32} /> : <Mic size={32} />}
            </button>
          </div>

          <div className=\"flex flex-col items-center gap-1 w-1/4\">
             <button
              onClick={() => setShowSettings(true)}
              className=\"p-3 rounded-full hover:bg-slate-100 text-slate-600 transition-colors\"
             >
              <Settings size={20} />
            </button>
            <span className=\"text-[10px] font-bold text-slate-400\">AYARLAR</span>
          </div>

        </div>
      </div>

    </div>
  );
}
"""
}

# Klasörleri oluştur
if not os.path.exists(project_name):
    os.makedirs(project_name)

# Dosyaları oluştur
for filepath, content in files.items():
    # Tam dosya yolunu oluştur
    full_path = os.path.join(project_name, filepath)

    # Alt klasör varsa oluştur
    os.makedirs(os.path.dirname(full_path), exist_ok=True)

    # Dosyayı yaz
    with open(full_path, "w", encoding="utf-8") as f:
        f.write(content)
        print(f"Oluşturuldu: {filepath}")

print(f"\n✅ Proje başarıyla oluşturuldu: {os.path.abspath(project_name)}")
print("Şimdi şu adımları takip edin:")
print(f"1. cd {project_name}")
print("2. npm install")
print("3. .env.local dosyasını açın ve API anahtarınızı girin.")
print("4. npm run dev")

Oluşturuldu: package.json
Oluşturuldu: tsconfig.json
Oluşturuldu: next.config.mjs
Oluşturuldu: tailwind.config.ts
Oluşturuldu: postcss.config.js
Oluşturuldu: next-env.d.ts
Oluşturuldu: .gitignore
Oluşturuldu: .env.local.example
Oluşturuldu: README.md
Oluşturuldu: app/globals.css
Oluşturuldu: app/layout.tsx
Oluşturuldu: app/page.tsx
Oluşturuldu: components/TranslatorApp.tsx

✅ Proje başarıyla oluşturuldu: /content/hollandali-translator
Şimdi şu adımları takip edin:
1. cd hollandali-translator
2. npm install
3. .env.local dosyasını açın ve API anahtarınızı girin.
4. npm run dev
